## Robustness.ipynb

This notebook will work with the [Robustness](https://github.com/MadryLab/robustness) Library from MadryLab.

In [1]:
!pip install robustness

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.3/95.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0

In [37]:
import sys
import torch.hub

# Define the function
def load_state_dict_from_url(*args, **kwargs):
    return torch.hub.load_state_dict_from_url(*args, **kwargs)

# Create a dummy module
class DummyModule:
    def __init__(self):
        self.load_state_dict_from_url = load_state_dict_from_url

# Replace the faulty import
sys.modules['torchvision.models.utils'] = DummyModule()

In [38]:
from robustness import model_utils, datasets, train, defaults
from robustness.datasets import CIFAR
import torch as ch

# We use cox (http://github.com/MadryLab/cox) to log, store and analyze
# results. Read more at https//cox.readthedocs.io.
from cox.utils import Parameters
import cox.store

In [39]:
from cox import utils
from cox import store

In [40]:
NUM_WORKERS = 16
BATCH_SIZE = 512
ds = CIFAR('./data')
m, _ = model_utils.make_and_restore_model(arch='resnet50', dataset=ds )
train_loader, val_loader = ds.make_loaders(batch_size=BATCH_SIZE, workers=NUM_WORKERS)

==> Preparing dataset cifar..
Files already downloaded and verified
Files already downloaded and verified


In [41]:
out_store = cox.store.Store('./')

Logging in: /content/5362610b-fd53-4d9c-9dc7-a393c4898883


In [42]:
# Hard-coded base parameters
train_kwargs = {
    'out_dir': "train_out",
    'adv_train': 1,
    'constraint': '2',
    'eps': 0.5,
    'attack_lr': 0.1,
    'attack_steps': 7,
    'epochs': 5
}
train_args = Parameters(train_kwargs)

# Fill whatever parameters are missing from the defaults
train_args = defaults.check_and_fill_args(train_args,
                        defaults.TRAINING_ARGS, CIFAR)
train_args = defaults.check_and_fill_args(train_args,  defaults.PGD_ARGS, CIFAR)

In [43]:
# train.train_model(train_args, m, (train_loader, val_loader), store=out_store)
pass

In [44]:
# get module names of m
m_names = []
for name, module in m.model._modules.items():
    m_names.append(name)
print(m.model)
print(m_names)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): SequentialWithArgs(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): Bottleneck(

In [55]:
import numpy as np

def activation_hook(module, input, output):
  activations.append(output.detach().cpu().numpy().reshape(output.shape[0], -1))
  # activations.append(input[0])
  return

In [56]:

activations = []
m.model.layer4.register_forward_hook(activation_hook)

with torch.no_grad():
  for images, labels in train_loader:
      images = images.cuda()
      labels = labels.cuda()
      m.eval()
      outputs = m(images)

      break
  # acts = np.concatenate(activations, axis=0)
  print(activations)
  print(activations[0].shape) #TODO: why are penultimate layer activations of dim 8192??

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[array([[ 0.0013232 ,  0.00069088, -0.00118969, ...,  0.00462793,
         0.00504268,  0.00441093],
       [ 0.00457542,  0.00273818,  0.00066063, ...,  0.01113324,
         0.010884  ,  0.00695054],
       [ 0.00220327,  0.00279534,  0.00325955, ...,  0.005388  ,
         0.00469452,  0.00543994],
       ...,
       [ 0.00038524,  0.00099583,  0.00068756, ...,  0.00231453,
         0.00235666,  0.00259897],
       [ 0.00156449,  0.00124717,  0.00206334, ...,  0.00682886,
         0.00493023,  0.00536594],
       [ 0.00057853,  0.00070501, -0.00055319, ...,  0.00304606,
         0.00272925,  0.0031471 ]], dtype=float32), array([[ 2.2548249e-03,  2.8227852e-03,  2.7954909e-03, ...,
         1.3088769e-03,  7.3310587e-04,  1.4083748e-03],
       [ 3.9108810e-03,  2.2582323e-03,  1.5041593e-03, ...,
         3.2244920e-04,  6.9038005e-04,  4.2978820e-05],
       [ 1.5750381e-03,  9.7034441e-04,  7.0263969e-04, ...,
         7.3420274e-04,  8.4731990e-04, -3.9222377e-04],
       ...,
    

In [50]:
acts.shape

(1024, 32768)